<a href="https://colab.research.google.com/github/staghado/Exploration-Notebooks/blob/main/Zephyr-7b-hf-open-llm-Mistral-7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick start to run Zephyr 7B Alpha on Google Colab

In [ ]:
# Install transformers from source - only needed for versions <= v4.34
%pip install git+https://github.com/huggingface/transformers.git
%pip install accelerate

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-r1fs6vb8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-r1fs6vb8
  Resolved https://github.com/huggingface/transformers.git to commit 34a640642be61e0e2eb622e497140be3aea202bc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7881983 sha256=3a11d8f014499f7c5d40ed72ace5b38805e4f0b7c575d28d3e2d99e33442757d
  Stored in directory: /tmp/pip-ephem-whee

## Load dependencies

In [ ]:
import torch
from transformers import pipeline

## Download model and load pipeline

In [ ]:
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## Prepare inputs

In [ ]:
# Each message can have 1 of 3 roles: "system" (to provide initial instructions), "user", or "assistant".
# For inference, make sure "user" is the role in the final message.
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who's objective is to be as helpful as possible. Always provide the answer as a JSON formatted output.",
    },
    {"role": "user", "content": """
    Given the following email, extract the company name, position title, application status. Do not output anything else.

Hi Said

Thank you for submitting your application for the Machine Learning Engineer role! We are thrilled to hear you are considering joining Criteo.

After reviewing your skills and experience, someone from our Talent Acquisition department will get back to you shortly. To review your submitted application and track its progress, you can log in to your candidate Criteo account here.

In the meantime, we invite you to get a sneak peek at Life at Criteo and the Criteo vibe.

Best of luck,

Criteo Talent Acquisition Team
"""},
]
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|system|>
You are a friendly chatbot who's objective is to be as helpful as possible. Always provide the answer as a JSON formatted output.</s>
<|user|>

    Given the following email, extract the company name, position title, application status. Do not output anything else.

Hi Said

Thank you for submitting your application for the Machine Learning Engineer role! We are thrilled to hear you are considering joining Criteo.

After reviewing your skills and experience, someone from our Talent Acquisition department will get back to you shortly. To review your submitted application and track its progress, you can log in to your candidate Criteo account here.

In the meantime, we invite you to get a sneak peek at Life at Criteo and the Criteo vibe.

Best of luck,

Criteo Talent Acquisition Team
</s>
<|assistant|>



## Generate!

In [ ]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot who's objective is to be as helpful as possible. Always provide the answer as a JSON formatted output.</s>
<|user|>

    Given the following email, extract the company name, position title, application status. Do not output anything else.

Hi Said

Thank you for submitting your application for the Machine Learning Engineer role! We are thrilled to hear you are considering joining Criteo.

After reviewing your skills and experience, someone from our Talent Acquisition department will get back to you shortly. To review your submitted application and track its progress, you can log in to your candidate Criteo account here.

In the meantime, we invite you to get a sneak peek at Life at Criteo and the Criteo vibe.

Best of luck,

Criteo Talent Acquisition Team
</s>
<|assistant|>
{
  "company name": "Criteo",
  "position title": "Machine Learning Engineer",
  "application status": "Application submitted and under review"
}


In [ ]:
d = eval(outputs[0]["generated_text"].split('<|assistant|>')[-1])
d

{'companyName': 'Criteo',
 'positionTitle': 'Machine Learning Engineer',
 'applicationStatus': 'Application reviewed, someone from Talent Acquisition department will get back to you shortly.'}